In [2]:
# Imports
from mmdet3d.apis import init_model, inference_detector
import time

# To convert to a .py file: jupyter nbconvert --to python model_evaluation.ipynb

In [1]:
# Configuration fields
DISPLAY_VISUALIZATION=True
DEVICE='cpu' # My machine doesn't have an NVIDIA graphics card, so set device to cpu so that we don't attemp to use CUDA cores
CONFIG_FILE='configs/pointpillars/hv_pointpillars_secfpn_6x8_160e_kitti-3d-3class.py'
CHECKPOINT='work_dirs/hv_pointpillars_secfpn_6x8_160e_kitti-3d-3class/epoch_10.pth'
INPUT_IMAGES=['data/cmpe249-fa22/kitti/testing/velodyne/000000.bin',
              '/home/hoch/dev/garage.bin', # Custom point cloud file
              'data/cmpe249-fa22/kitti/testing/velodyne/000001.bin',
              'data/cmpe249-fa22/kitti/testing/velodyne/000002.bin',
              'data/cmpe249-fa22/kitti/testing/velodyne/000003.bin',
              'data/cmpe249-fa22/kitti/testing/velodyne/000004.bin',
              'data/cmpe249-fa22/kitti/testing/velodyne/000005.bin',
              'data/cmpe249-fa22/kitti/testing/velodyne/000006.bin',
              'data/cmpe249-fa22/kitti/testing/velodyne/000007.bin',
              'data/cmpe249-fa22/kitti/testing/velodyne/000008.bin',
              'data/cmpe249-fa22/kitti/testing/velodyne/000009.bin']

In [5]:
# Initialize the model using the given config file and checkpoint pkl
print('------------------------------')
print('Initializing model using the following..')
print(f'Config file:     {CONFIG_FILE}')
print(f'Checkpoint file: {CHECKPOINT}')
print(f'Device:          {DEVICE}')
print('------------------------------')
model = init_model(CONFIG_FILE, CHECKPOINT, DEVICE)

num_images = len(INPUT_IMAGES)
print(f'Number of inference images on: {num_images}')

time_deltas = []
for image in INPUT_IMAGES:
    print(f'Performing inference on {image}..')

    start_time = time.time()
    results, data = inference_detector(model, image)
    time_delta = time.time() - start_time
    
    print(f'Completed, latency of {time_delta}s')
    time_deltas.append(time_delta)

    # model.show_results(data, results, out_dir='results', show=DISPLAY_VISUALIZATION)

inference_time_total = sum(time_deltas)
print('------------------------------')
print(f'Total inference latency:   {inference_time_total}s')
print(f'Average inference latency: {inference_time_total / num_images}s')

2022-09-08 01:57:02,505 - mmdet3d - WARNING - Don't suggest using CPU device. Some functions are not supported for now.
WARNING - 2022-09-08 01:57:02,505 - inference - Don't suggest using CPU device. Some functions are not supported for now.


------------------------------
Initializing model using the following..
Config file:     configs/pointpillars/hv_pointpillars_secfpn_6x8_160e_kitti-3d-3class.py
Checkpoint file: work_dirs/hv_pointpillars_secfpn_6x8_160e_kitti-3d-3class/epoch_10.pth
Device:          cpu
------------------------------
load checkpoint from local path: work_dirs/hv_pointpillars_secfpn_6x8_160e_kitti-3d-3class/epoch_10.pth
Number of inference images on: 11
Performing inference on data/cmpe249-fa22/kitti/testing/velodyne/000000.bin..
Completed, latency of 0.5554642677307129s
Performing inference on /home/hoch/dev/garage.bin..


IndexError: index -1 is out of bounds for dimension 0 with size 0

In [ ]:
import open3d as o3d
import numpy as np
import torch
import csv
import os

pcd_file_path='/home/hoch/dev/pcd2bin/pcd/garage_color.pcd'
csv_file_path='/home/hoch/dev/pcd2bin/pcd/meta.csv'
bin_file_path='/home/hoch/dev/garage.bin'
pcd_file = o3d.io.read_point_cloud(pcd_file_path)


# pcd_file_path='/home/hoch/dev/pcd2bin/pcd/garage.pcd'
# pcd_file=pypcd.PointCloud.from_path(pcd_file_path)
# print(pcd_file_path)
# print(pcd_file.__dict__)

# pcd_file_path='/home/hoch/dev/pcd2bin/pcd/garage_color.pcd'
# pcd_file=pypcd.PointCloud.from_path(pcd_file_path)
# print(pcd_file_path)
# print(pcd_file.__dict__)

In [ ]:
print(np.asarray(pcd_file.points).shape)
o3d.visualization.draw_geometries([pcd_file])

print("Downsample the point cloud with a voxel of 0.05")
downpcd = pcd_file.voxel_down_sample(voxel_size=0.05)
o3d.visualization.draw_geometries([downpcd])

print("Recompute the normal of the downsampled point cloud")
downpcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(
    radius=0.1, max_nn=30))
o3d.visualization.draw_geometries([downpcd])

In [ ]:
new_pcd = o3d.t.io.read_point_cloud(pcd_file_path)
positions  = new_pcd.point['positions'].numpy().astype(np.float32)
colors     = new_pcd.point['colors'].numpy().astype(np.float32)
# normals = new_pcd.point['normals']


In [ ]:
x_values = positions[:, 0]
y_values = positions[:, 1]
z_values = positions[:, 2]
# https://stackoverflow.com/questions/687261/converting-rgb-to-grayscale-intensity#comment20506224_689547
# Y = 0.2126 * R + 0.7152 * G + 0.0722 * B
l_values = 0.2126 * colors[:, 0] + \
           0.7152 * colors[:, 1] + \
           0.0722 * colors[:, 2]

# i_values = (116 * l_values**(1/3)) - 16
i_values = l_values / 255.0

new_points = np.transpose(np.vstack((x_values, y_values, z_values, i_values)))
print(new_points.shape)
new_points = new_points[0:125635]
print(new_points.shape)
new_points.tofile(bin_file_path)

# 503304
# 502540
# pc = pypcd.PointCloud.from_msg(msg)
# x = pc.pc_data['x']
# y = pc.pc_data['y']
# z = pc.pc_data['z']
# intensity = pc.pc_data['intensity']

In [ ]:
bad_file  = np.fromfile(INPUT_IMAGES[1], dtype='float32')
good_file = np.fromfile(INPUT_IMAGES[0], dtype='float32')

In [ ]:
for image in INPUT_IMAGES:
    current_file = np.fromfile(image, dtype='float32')
    print(f'image: {image} -- shape: {current_file.shape}')
    print(current_file[0:4])